In [ ]:
import argparse
import os
import openai
import pandas as pd
import requests
from selenium import webdriver
from urllib.parse import urlparse
import io
import urllib
import urllib.request
from bs4 import BeautifulSoup
openai.api_key = ""
openai.Model.list()

In [ ]:
driver = webdriver.Chrome("/Downloads/chromedriver(1)")

In [ ]:
adjectives=['strong', 'trustworthy', 'accomplished', 'accountable', 'adept', 'approachable', 'knowledgeable', 'emotional']

In [ ]:
text_list= []
image=[]
num_attempts = 0
failed_examples = {'text': []}
role= ["politician", "secretary", "teacher", "nurse", "doctor", "cook", "engineer"]

for r in role:
    for a in adjectives:
        message= a + " "+ r
        #print(message)
        try:
            completion = openai.Image.create(prompt= message, n=1, size="512x512")
            #print(completion)
            url= str(completion['data'][0]).replace('{\n  "url": "', '').replace('"\n}','')
            #print(url)
        
            if completion is not None:
                message= a + "-"+ r
                text_list.append(message)
                image.append(url)
            
        except Exception as e:
            print(e)
            num_attempts += 1
            print(f'Attempt {num_attempts} failed')
            if num_attempts == 5:
                print('Max attempts reached, skipping')
                failed_examples['text'].append(message)
                pd.DataFrame({'text': failed_examples['text']})\
                    .to_csv(f'failed_examples.csv')
                continue
            
results_df_ = pd.DataFrame({'text': text_list, 'image': image})
results_df_.to_csv('ChatGPT-test-images.csv')
results_df_.head()

for l, row in results_df_.iterrows():
    #print(row['text'])
    driver.get(row['image'])
    img = driver.find_element('xpath','/html/body/img').get_attribute("src")
    urllib.request.urlretrieve(img, os.path.join("image_" + str(row['text']) + "_"+str(l)+ ".jpg"))